In [ ]:
import argparse
import os
import pathlib
import sys
import time

import psutil

sys.path.append("../featurization_utils")
import cucim.skimage.measure
import cucim.skimage.segmentation
import cupy
import cupyx.scipy.ndimage
import numpy as np
import pandas as pd
import scipy
import skimage
from intensity_utils import measure_3D_intensity_gpu
from loading_classes import ImageSetLoader, ObjectLoader
from resource_profiling_util import get_mem_and_time_profiling

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

In [ ]:
if not in_notebook:
    argparser = argparse.ArgumentParser()
    argparser.add_argument(
        "--well_fov",
        type=str,
        default="None",
        help="Well and field of view to process, e.g. 'A01_1'",
    )

    args = argparser.parse_args()
    well_fov = args.well_fov
    if well_fov == "None":
        raise ValueError(
            "Please provide a well and field of view to process, e.g. 'A01_1'"
        )

    image_set_path = pathlib.Path(f"../../data/NF0014/cellprofiler/{well_fov}/")
else:
    well_fov = "C4-2"
    image_set_path = pathlib.Path(f"../../data/NF0014/cellprofiler/{well_fov}/")

In [3]:
channel_n_compartment_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [4]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    anisotropy_spacing=(1, 0.1, 0.1),
    channel_mapping=channel_n_compartment_mapping,
)

In [ ]:
start_time = time.time()
# get starting memory (cpu)
start_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [6]:
for compartment in tqdm(
    image_set_loader.compartments, desc="Processing compartments", position=0
):
    for channel in tqdm(
        image_set_loader.image_names,
        desc="Processing channels",
        leave=False,
        position=1,
    ):
        object_loader = ObjectLoader(
            image_set_loader.image_set_dict[channel],
            image_set_loader.image_set_dict[compartment],
            channel,
            compartment,
        )

        output_dict = measure_3D_intensity_gpu(object_loader)
        final_df = pd.DataFrame(output_dict)
        # prepend compartment and channel to column names
        final_df = final_df.pivot(
            index=["object_id"],
            columns="feature_name",
            values="value",
        ).reset_index()
        for col in final_df.columns:
            if col == "object_id":
                continue
            else:
                final_df.rename(
                    columns={col: f"Intensity_{compartment}_{channel}_{col}"},
                    inplace=True,
                )

        final_df.insert(0, "image_set", image_set_loader.image_set_name)

        output_file = pathlib.Path(
            f"../results/{image_set_loader.image_set_name}/Intensity_{compartment}_{channel}_features.parquet"
        )
        output_file.parent.mkdir(parents=True, exist_ok=True)
        final_df.to_parquet(output_file)

/home/lippincm/miniforge3/envs/GFF_featurization/lib/python3.12/site-packages/cupyx/scipy/ndimage/_measurements.py:397: PerformanceWarning: Using the slower implementation as cupyx.scipy.ndimage.sum supports int32, float16, float32, float64, uint32, uint64 as data typesfor the fast implmentation
  warnings.warn(
/home/lippincm/miniforge3/envs/GFF_featurization/lib/python3.12/site-packages/cupyx/scipy/ndimage/_measurements.py:487: PerformanceWarning: Using the slower implementation as cupyx.scipy.ndimage.mean supports int32, float16, float32, float64, uint32, uint64 as data types for the fast implmentation
  warnings.warn(


In [ ]:
end_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2
end_time = time.time()
get_mem_and_time_profiling(
    start_mem=start_mem,
    end_mem=end_mem,
    start_time=start_time,
    end_time=end_time,
    process_name="Intensity",
    well_fov=well_fov,
    CPU_GPU="GPU",
)

Intensity time:
--- 85.91671085357666 seconds ---
--- 1.4319456497828165 minutes ---
--- 0.023865768710772197 hours ---
